In [13]:
import random
import numpy as np
import pandas as pd
import utilization_waiting as ut
from time import clock

clock()
nurse = 8
doctor = 4
patient = random.randint(15, 25)  # random variable
patient_nurse = int(patient * 0.2)  # patients only to see nurse
patient_nurse_doctor = patient - patient_nurse  # patients to see nurse & doctor

# (mean, sd, min, max) random variables indicating the time(minutes) a patient needs to spent with a nurse/doctor
mean_time = random.randint(25, 35)
sd = 1
min_time = mean_time - 10
max_time = mean_time + 10

# nurse only
total_nurseonly_time = ut.sum_truncnorm(mean_time, sd, min_time, max_time, patient_nurse)
# nurse and doctor
total_nursemix_time = ut.sum_truncnorm(mean_time, sd, min_time, max_time, patient_nurse_doctor)
total_doctor_time = ut.sum_truncnorm(mean_time, sd, min_time, max_time, patient_nurse_doctor)

# utilization
utilization_nurse = ut.rate((total_nurseonly_time + total_nursemix_time), nurse)
utilization_doctor = ut.rate((total_doctor_time), doctor)
print("\nThe utilization of nurse is {}%".format(round(utilization_nurse * 100), 2))
print("The utilization of doctor is {}%".format(round(utilization_doctor * 100), 2))


list_patient = []
for i in range(patient):
    list_patient.append('patient_{}'.format(i+1))

total_time = 60
random_arrival = [random.random() for i in range(patient)]
sum_arrival = sum(random_arrival)
arrival_time = [total_time * i / sum_arrival for i in random_arrival]
print(arrival_time)

# arrive_time = float(60/patient)

df_patients = pd.DataFrame({'Arrival_point': 0, 'Nurse_only': False,
                            'Waiting_nurse': 0, 'Meet_nurse_point': 0,
                            'Seeing_nurse': 0, 'After_nurse_point': 0,
                            'Waiting_doctor': 0, 'Meet_doctor_point': 0,
                            'Seeing_doctor': 0, 'After_doctor_point': 0}, index=list_patient)

for i in range(patient):
    df_patients.loc[list_patient[i], 'Arrival_point'] += sum(arrival_time[:i])

list_nurseonly_number = random.sample(list(range(patient)), patient_nurse)
for i in list_nurseonly_number:
    df_patients.loc['patient_{}'.format(i + 1), 'Nurse_only'] = True

arr_nurse = np.zeros(nurse)
arr_doctor = np.zeros(doctor)

df_patients['Seeing_nurse'] = ut.time_spend(mean_time, sd, min_time, max_time, patient)

for i in range(patient):
    if min(arr_nurse) <= df_patients.loc[list_patient[i], 'Arrival_point']:
        df_patients.loc[list_patient[i], 'Meet_nurse_point'] = df_patients.loc[list_patient[i], 'Arrival_point']
    else:
        df_patients.loc[list_patient[i], 'Meet_nurse_point'] = min(arr_nurse)
    df_patients.loc[list_patient[i], 'Waiting_nurse'] = df_patients.loc[list_patient[i], 'Meet_nurse_point'] - df_patients.loc[list_patient[i], 'Arrival_point']
    df_patients.loc[list_patient[i], 'After_nurse_point'] = df_patients.loc[list_patient[i], 'Meet_nurse_point'] + df_patients.loc[list_patient[i], 'Seeing_nurse']

    arr_nurse[np.argmin(arr_nurse)] += df_patients.loc[list_patient[i], 'Seeing_nurse']


list_mix_patient = [p for p in df_patients[df_patients['Nurse_only'] == False].index]
time_doctor = ut.time_spend(mean_time, sd, min_time, max_time, patient_nurse_doctor)
for i in range(patient_nurse_doctor):
    df_patients.loc[list_mix_patient[i], 'Seeing_doctor'] = time_doctor[i]

for i in range(patient_nurse_doctor):
    if min(arr_doctor) <= df_patients.loc[list_mix_patient[i], 'After_nurse_point']:
        df_patients.loc[list_mix_patient[i], 'Meet_doctor_point'] = df_patients.loc[list_mix_patient[i], 'After_nurse_point']
    else:
        df_patients.loc[list_mix_patient[i], 'Meet_doctor_point'] = min(arr_doctor)
    df_patients.loc[list_mix_patient[i], 'Waiting_doctor'] = df_patients.loc[list_mix_patient[i], 'Meet_doctor_point'] - df_patients.loc[list_mix_patient[i], 'After_nurse_point']
    arr_doctor[np.argmin(arr_doctor)] += df_patients.loc[list_mix_patient[i], 'Seeing_doctor']
    df_patients.loc[list_mix_patient[i], 'After_doctor_point'] = df_patients.loc[list_mix_patient[i], 'Meet_doctor_point'] + df_patients.loc[list_mix_patient[i], 'Seeing_doctor']


average_wait_nurse = round(df_patients['Waiting_nurse'].mean(), 2)
max_wait_nurse = round(df_patients['Waiting_nurse'].max(), 2)
average_wait_doctor = round(df_patients['Waiting_doctor'].sum() / patient_nurse_doctor, 2)
max_wait_doctor = round(df_patients['Waiting_doctor'].max(), 2)

print('\nThe average and maxium waiting time for nurse are {} minutes and {} minutes'.format(average_wait_nurse, max_wait_nurse))
print('The average and maxium waiting time for doctor are {} minutes and {} minutes'.format(average_wait_doctor, max_wait_doctor))

print('\nTotal running time is %-1.5ss' % clock())

df_patients


The utilization of nurse is 141.0%
The utilization of doctor is 230.0%
[4.202566524693255, 2.3583778668764306, 0.2860022537111618, 1.6962043859695537, 1.487729711701985, 4.044156946145485, 4.159513625264958, 2.0100603844410876, 3.4751975656625866, 3.962495717613746, 5.164827939014717, 4.943697248570227, 2.0436633258145034, 3.9733324747263694, 4.21920009546088, 3.1685111962373265, 0.8528513697925213, 2.5301587227539963, 0.8124769586006754, 2.397822067912973, 2.21115361903556]

The average and maxium waiting time for nurse are 3.13 minutes and 10.69 minutes
The average and maxium waiting time for doctor are 4.36 minutes and 31.62 minutes

Total running time is 3.193s


,Arrival_point,Nurse_only,Waiting_nurse,Meet_nurse_point,Seeing_nurse,After_nurse_point,Waiting_doctor,Meet_doctor_point,Seeing_doctor,After_doctor_point
patient_1,0.000000,False,0.000000,0.000000,31.641409,31.641409,0.000000,31.641409,31.464363,63.105771
patient_2,4.202567,False,0.000000,4.202567,30.937493,35.140060,0.000000,35.140060,33.373907,68.513967
patient_3,6.560944,False,0.000000,6.560944,31.128414,37.689358,0.000000,37.689358,32.761477,70.450836
patient_4,6.846947,False,0.000000,6.846947,32.172254,39.019201,0.000000,39.019201,31.275959,70.295160
patient_5,8.543151,False,0.000000,8.543151,32.141854,40.685005,0.000000,40.685005,32.359494,73.044498
patient_6,10.030881,False,0.000000,10.030881,32.162347,42.193228,0.000000,42.193228,33.158963,75.352190
patient_7,14.075038,False,0.000000,14.075038,33.789248,47.864286,0.000000,47.864286,31.834931,79.699216
patient_8,18.234551,True,0.000000,18.234551,31.537447,49.771999,0.000000,0.000000,0.000000,0.000000
patient_9,20.244612,False,10.692882,30.937493,32.940543,63.878036,0.000000,63.878036,30.794384,94.672420
patient_10,23.719809,False,7.408605,31.128414,30.577698,61.706112,1.929341,63.635453,32.786507,96.421961
